In [1]:
# Import dependencies
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from datetime import datetime, date

In [2]:
# Import configuration variables
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
# Set up simple_salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
# Connect to your MySQL database
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Query 'staff' table from MySql

In [5]:
# Query Data
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staff_data.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [7]:
# Transform Data to perfectly match between dataframe headers and field names in Salesforce
staff_data = staff_data.rename(columns={
                                        'ID_Staff':'ID_Staff__c',
                                        'EmployeeID':'EmployeeID__c',
                                        'LastName':'Last_Name__c',
                                        'FirstName':'Name',
                                        'MiddleName':'Middle__c',
                                        'BirthDate':'Birth_Date__c'                                     
                                })
staff_data.head()

,ID_Staff__c,EmployeeID__c,Last_Name__c,Name,Middle__c,Birth_Date__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [8]:
# Change to readable form 
staff_data['Birth_Date__c'] = staff_data['Birth_Date__c'].apply(lambda x: x.strftime('%Y-%m-%d')) 
staff_data

,ID_Staff__c,EmployeeID__c,Last_Name__c,Name,Middle__c,Birth_Date__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [9]:
# Convert DataFrame to a list of dictionaries --> This can help facilitate a bulk insert
staff_load = staff_data.to_dict(orient='record')
staff_load

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'Last_Name__c': 'Wimberly',
  'Name': 'Sam',
  'Middle__c': 'Nico',
  'Birth_Date__c': '1995-07-05'},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'Name': 'Gemini',
  'Middle__c': 'Blair',
  'Birth_Date__c': '1992-04-22'},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'Last_Name__c': 'Williams',
  'Name': 'Dartanion',
  'Middle__c': 'De Angelo',
  'Birth_Date__c': '1993-05-21'},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'Last_Name__c': 'Popelka',
  'Name': 'Sarah',
  'Middle__c': 'Nisan',
  'Birth_Date__c': '1994-06-07'}]

In [12]:
'''
# Inserting rows in bulk for Class table on Salesforce
try:
    sf.bulk.Staff__c.insert(staff_load)
except Exception as e:
    print(e)
'''

'\n# Inserting rows in bulk for Class table on Salesforce\ntry:\n    sf.bulk.Staff__c.insert(staff_load)\nexcept Exception as e:\n    print(e)\n'

In [11]:
for row in staff_load:

    record = {
        'ID_Staff__c': row['ID_Staff__c'],
        'EmployeeID__c': row['EmployeeID__c'],
        'Last_Name__c': row['Last_Name__c'],
        'Name':row['Name'],
        'Middle__c':row['Middle__c'],
        'Birth_Date__c':row['Birth_Date__c']
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)